# Exploratory Data Analysis (EDA) – FDI Panel

This notebook explores the cleaned FDI panel datasets created in the data preparation step.

**Inputs (from `1_data_preparation/`):**
- `panel_innerjoin_strict.csv` – balanced panel, complete data (for clean comparisons)
- `panel_leftjoin_code_only.csv` – unbalanced panel, maximum coverage (for broad exploration)

Goals of this notebook:
- Understand the structure of the data
- Inspect basic distributions and ranges
- Check missingness patterns
- Visualize simple relationships between:
  - FDI and macroeconomic indicators
  - FDI and governance / infrastructure / human capital
- Explore time trends by year and by country

No inferential statistics or machine learning are performed here – this is **pure exploration**.


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Plot settings
plt.style.use("default")
plt.rcParams["figure.figsize"] = (10, 6)

# Display options
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 180)


## 1. Load Datasets

We load both versions of the panel:

- `panel_inner` – inner join on (Country, Code, Year): **strict, balanced**  
- `panel_left` – left join on Code (with year): **unbalanced, wider coverage**

File paths assume this notebook lives in `2_data_exploration/`.


In [59]:
import pandas as pd
panel_inner = pd.read_csv("ELO_2/panel_regression_inner_join.csv")
panel_left  = pd.read_csv("ELO_2/panel_full_left_join.csv")

panel_inner.head()



,Country,Code,Year,Income_group,FDI_inflows,GDP_current_USD,GDP_growth,Trade_pct_GDP,Inflation_CPI,Electricity_access,Education_enrollment,LPI_score,CPI
0,Albania,ALB,2012,Upper middle income,9.179946e+08,1.224650e+10,0.984130,76.968358,2.031593,99.9,97.608612,2.77,33.0
1,Azerbaijan,AZE,2012,Upper middle income,5.293250e+09,6.967994e+10,2.202939,78.263070,1.066213,100.0,82.762275,2.48,27.0
2,Burundi,BDI,2012,Low income,6.049197e+05,2.333341e+09,4.446706,43.705373,18.161045,6.5,29.573290,1.61,19.0
3,Burkina Faso,BFA,2012,Low income,3.292820e+08,1.256102e+10,6.452672,61.238608,3.818152,15.1,25.239968,2.32,38.0
4,Bosnia and Herzegovina,BIH,2012,Upper middle income,3.919769e+08,1.722674e+10,-0.821836,88.145109,2.052675,99.7,90.785309,2.99,42.0


In [61]:
print("INNER PANEL shape:", panel_inner.shape)
print("LEFT  PANEL shape:", panel_left.shape)

print("\nColumns:")
print(panel_inner.columns.tolist())


INNER PANEL shape: (218, 13)
LEFT  PANEL shape: (403, 13)

Columns:
['Country', 'Code', 'Year', 'Income_group', 'FDI_inflows', 'GDP_current_USD', 'GDP_growth', 'Trade_pct_GDP', 'Inflation_CPI', 'Electricity_access', 'Education_enrollment', 'LPI_score', 'CPI']


In [ ]:
## 2. Basic Structure & Data Types

Here we check:
- column data types
- basic non-null counts
- uniqueness and key structure


In [ ]:
print("=== panel_inner.info() ===")
panel_inner.info()

print("\n=== Key uniqueness check (Country, Code, Year) in INNER panel ===")
print(panel_inner[["Country", "Code", "Year"]].duplicated().any())
print("Duplicated rows:", panel_inner[["Country", "Code", "Year"]].duplicated().sum())


In [ ]:
numeric_cols = panel_inner.select_dtypes(include=[np.number]).columns
panel_inner[numeric_cols].describe().T


## 3. Missing Data Overview

We examine:
- overall missingness per column
- a simple missingness heatmap for a subset of variables


In [ ]:
subset_cols = ["FDI_inflows", "GDP_current_USD", "GDP_growth",
               "Trade_pct_GDP", "Inflation_CPI",
               "Electricity_access", "Education_enrollment",
               "LPI_score", "CPI"]

subset_cols = [c for c in subset_cols if c in panel_inner.columns]

plt.figure(figsize=(10, 5))
sns.heatmap(panel_inner[subset_cols].isna(), cbar=False)
plt.title("Missing Data Pattern – Selected Variables (panel_inner)")
plt.xlabel("Variables")
plt.ylabel("Observations")
plt.tight_layout()
plt.show()


## 4. Distributions of Key Variables

We look at simple histograms / KDE plots for:
- FDI inflows
- GDP (size)
- Trade openness
- Inflation
- Education, electricity, LPI, CPI (if present)


In [ ]:
def plot_hist(col, df=panel_inner, bins=30):
    if col not in df.columns:
        print(f"[skip] {col} not in dataframe.")
        return
    plt.figure()
    sns.histplot(df[col].dropna(), bins=bins, kde=True)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()

for col in ["FDI_inflows", "GDP_current_USD", "GDP_growth",
            "Trade_pct_GDP", "Inflation_CPI",
            "Electricity_access", "Education_enrollment",
            "LPI_score", "CPI"]:
    plot_hist(col)


## 5. Correlation Structure (Descriptive Only)

Here we compute and visualize pairwise correlations between numeric variables.

This is **descriptive**, not causal:
- helps to see which variables move together
- suggests which variables might be interesting in later modeling


In [ ]:
corr = panel_inner[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, cmap="coolwarm", center=0, annot=False)
plt.title("Correlation Matrix – Numeric Variables (panel_inner)")
plt.tight_layout()
plt.show()

corr["FDI_inflows"].sort_values(ascending=False)


## 6. Time Trends

We now look at how key variables evolve between 2010 and 2024.

Examples:
- Average FDI inflows by year
- Median GDP growth by year


In [ ]:
# Make sure Year is numeric
panel_inner["Year"] = pd.to_numeric(panel_inner["Year"], errors="coerce")

group_year = panel_inner.groupby("Year")

agg_year = group_year[["FDI_inflows", "GDP_current_USD", "GDP_growth",
                       "Trade_pct_GDP", "Inflation_CPI"]].agg(["mean", "median"])

agg_year


In [ ]:
plt.figure()
group_year["FDI_inflows"].mean().plot(marker="o")
plt.title("Average FDI Inflows over Time")
plt.ylabel("FDI_inflows (mean)")
plt.xlabel("Year")
plt.tight_layout()
plt.show()

plt.figure()
group_year["GDP_growth"].median().plot(marker="o")
plt.title("Median GDP Growth over Time")
plt.ylabel("GDP_growth (median)")
plt.xlabel("Year")
plt.tight_layout()
plt.show()


## 7. Country-Level Profiles (Template)

We create a simple function to plot:
- FDI inflows over time
- One or two other indicators

You can call this function for any country code.


In [ ]:
def plot_country_profile(code, df=panel_inner):
    if code not in df["Code"].unique():
        print(f"Code {code} not found.")
        return
    
    sub = df[df["Code"] == code].sort_values("Year")
    country_name = sub["Country"].iloc[0]
    
    fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
    
    # FDI inflows
    axes[0].plot(sub["Year"], sub["FDI_inflows"], marker="o")
    axes[0].set_title(f"{country_name} ({code}) – FDI Inflows")
    axes[0].set_ylabel("FDI_inflows")
    
    # Example: GDP growth (if present)
    if "GDP_growth" in sub.columns:
        axes[1].plot(sub["Year"], sub["GDP_growth"], marker="o", color="taborange")
        axes[1].set_title(f"{country_name} ({code}) – GDP Growth")
        axes[1].set_ylabel("GDP_growth")
    else:
        axes[1].text(0.5, 0.5, "GDP_growth not available", ha="center")
    
    axes[1].set_xlabel("Year")
    plt.tight_layout()
    plt.show()

# Example calls (change codes as you like)
plot_country_profile("ETH")  # Ethiopia (if present)
plot_country_profile("CHN")  # China
plot_country_profile("IND")  # India


## 8. Next Steps

From this EDA we should document:

- Variables with serious missingness  
- Obvious outliers to consider (log transforms, winsorizing, or exclusion)  
- Indicators that show promising relationships with FDI inflows  
- Any suspicious values (e.g., negative or zero FDI, impossible growth rates)  

These notes will guide:
- Feature selection
- Transformations (e.g., log(FDI), log(GDP))
- Modeling strategies in the analysis phase
